# **Aprenizagem de Máquina - Exemplo um-para-um**

## Demonstração 1 <br>
Treinando um modelo para prever verbos e substantivos


### Importando as dependências

In [ ]:
import torch
import torch.nn as nn
from torch import optim

### Dados <br>
Redes Neurais lidam com números. Portanto, as strings devem ser convertidas em índices, tanto os nomes quanto as classes.

In [ ]:
#dados rotulados
X = [
    {'X': 'amar', 'y': 'verbo'},
    {'X': 'Ándre', 'y': 'substantivo'},
    {'X': 'beber', 'y': 'verbo'},
    {'X': 'Bárbara', 'y': 'substantivo'},
    {'X': 'Danilo', 'y': 'substantivo'},
    {'X': 'menino', 'y': 'verbo'},
    {'X': 'mulher', 'y': 'verbo'},
    {'X': 'sorrir', 'y': 'verbo'},
    {'X': 'Vivian', 'y': 'substantivo'},
    {'X': 'viver', 'y': 'verbo'},
]

#Mapping de classe para índice e índice para classe
c2id = {'substantivo': 0, 'verbo': 1}
id2c = {0: 'substantivo', 1: 'verbo'}

#Mapping de nome para índice e índice para nome
x2id = {w['X']:i for i, w in enumerate(X)}
#criação de um símbolo OOV (out of vocabulary) para nomes fora do conjunto de treinamento
x2id['oov'] = len(x2id)
id2x = {i:w for (w, i) in x2id.items()}
x2id

{'amar': 0,
 'Ándre': 1,
 'beber': 2,
 'Bárbara': 3,
 'Danilo': 4,
 'menino': 5,
 'mulher': 6,
 'sorrir': 7,
 'Vivian': 8,
 'viver': 9,
 'oov': 10}

### Modelo <br>
Desenvolvido com PyTorch

In [ ]:
class Predictor(nn.Module):
  def __init__(self, inp_dim, n_classes, x2id):
    super(Predictor, self).__init__()
    self.x2id = x2id
    self.lookup = nn.Embedding(len(x2id), inp_dim)
    self.Wb = nn.Linear(inp_dim, n_classes)
    self.softmax = nn.LogSoftmax(1)

  def forward(self, X):
    idxs = []
    for x in X:
      try:
        idxs.append(self.x2id[x])
      except:
        idxs.append(self.x2id['oov'])
    idxs = torch.tensor(idxs)
    embeddings = self.lookup(idxs)
    z = self.Wb(embeddings)
    return self.softmax(z)

### Instanciando o modelo

In [ ]:
inp_dim = 3
n_classes = len(c2id)
model = Predictor(inp_dim, n_classes, x2id)

### Instanciando função de erro, otimizador e dados em lotes

In [ ]:
nepochs = 10
batch_size = 2
batch_status = 2
learning_rate = 0.1
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
from torch.utils.data import DataLoader, Dataset

traindata = DataLoader(X, batch_size=batch_size, shuffle=True)

### Treinamento

In [ ]:
for epoch in range(nepochs):
  losses = []
  for batch_idx, row in enumerate(traindata):
    X = row['X']
    y = torch.tensor([c2id[c] for c in row['y']])

    #forward
    outputs = model(X)

    #calculate loss
    loss = criterion(outputs, y)
    losses.append(float(loss))

    #backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    #diplay
    if (batch_idx+1) % batch_status == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(
                                                                                  epoch+1, batch_idx + 1, len(traindata),
                                                                                  100. * batch_idx / len(traindata), float(loss),
                                                                                  round(sum(losses) / len(losses), 5)))

Train Epoch: 1 [2/5 (20%)]	Loss: 0.796322	Total Loss: 0.713820
Train Epoch: 1 [4/5 (60%)]	Loss: 0.921497	Total Loss: 0.679870
Train Epoch: 2 [2/5 (20%)]	Loss: 0.216533	Total Loss: 0.280030
Train Epoch: 2 [4/5 (60%)]	Loss: 0.132326	Total Loss: 0.269270
Train Epoch: 3 [2/5 (20%)]	Loss: 0.222173	Total Loss: 0.231490
Train Epoch: 3 [4/5 (60%)]	Loss: 0.024899	Total Loss: 0.135900
Train Epoch: 4 [2/5 (20%)]	Loss: 0.016874	Total Loss: 0.019080
Train Epoch: 4 [4/5 (60%)]	Loss: 0.007456	Total Loss: 0.022890
Train Epoch: 5 [2/5 (20%)]	Loss: 0.003020	Total Loss: 0.003880
Train Epoch: 5 [4/5 (60%)]	Loss: 0.008095	Total Loss: 0.004810
Train Epoch: 6 [2/5 (20%)]	Loss: 0.001627	Total Loss: 0.001390
Train Epoch: 6 [4/5 (60%)]	Loss: 0.004090	Total Loss: 0.002220
Train Epoch: 7 [2/5 (20%)]	Loss: 0.000761	Total Loss: 0.000890
Train Epoch: 7 [4/5 (60%)]	Loss: 0.000825	Total Loss: 0.001030
Train Epoch: 8 [2/5 (20%)]	Loss: 0.001024	Total Loss: 0.000570
Train Epoch: 8 [4/5 (60%)]	Loss: 0.001550	Total Loss: 0

### Predição

In [ ]:
val_X = ["jogar", "amar", "cantar", "Vivian"]

outputs = model(val_X)
val_y = torch.argmax(outputs, 1).tolist()

[id2c[c] for c in val_y]

['verbo', 'verbo', 'verbo', 'substantivo']

## Demonstração 2 <br>
Treinando um modelo para prever se um nome é masculino ou feminino

### Importando dependências

In [ ]:
import torch
import torch.nn as nn
from torch import optim
import nltk
from nltk.corpus import names
nltk.download()

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

### Dados

In [ ]:
from random import shuffle
masc = names.words('male.txt')
fem = names.words('female.txt')

data = [{'X': row, 'y': 'male'} for row in masc]
data += [{'X': row, 'y': 'female'} for row in fem]

name2id = {}
pos = 0
for name in data:
  if name['X'] not in name2id:
    name2id[name['X']] = pos
    pos += 1
name2id['oov'] = len(name2id.keys())
id2name = { i:name for (name, i) in name2id.items() }

c2id = {'male': 0, 'female': 1}
id2c = {0: 'male', 1: 'female'}

### Separando dados de treino e teste

In [ ]:
shuffle(data)

size = int(len(data) * 0.2)
trainset = data[size:]
testset = data[:size]

### Modelo: Multi-layer Perceptron

In [ ]:
class NamePredictor(nn.Module):
  def __init__(self, inp_dim, hdim, n_classes, x2id):
    super(NamePredictor, self).__init__()
    self.x2id = x2id
    self.lookup = nn.Embedding(len(x2id), inp_dim)
    self.layer1 = nn.Linear(inp_dim, hdim)
    self.activation1 = nn.Sigmoid()
    self.layer2 = nn.Linear(hdim, n_classes)
    self.softmax = nn.LogSoftmax(1)

  def forward(self, X):
    idxs = []
    for x in X:
      try:
        idxs.append(self.x2id[x])
      except:
        idxs.append(self.x2id['oov'])
    idxs = torch.tensor(idxs)
    embeddings = self.lookup(idxs)

    z1 = self.layer1(embeddings)
    a1 = self.activation1(z1)
    z2 = self.layer2(a1)
    return self.softmax(z2)

### Inicializando o modelo

In [ ]:
inp_dim = 3
hdim = 10
n_classes = len(c2id)
model = NamePredictor(inp_dim, hdim, n_classes, name2id)

### Instanciando função de erro, otimizadores e dados em lote

In [ ]:
nepochs = 20
batch_size = 16
batch_status = 64
learning_rate = 0.01
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
from torch.utils.data import DataLoader, Dataset

traindata = DataLoader(trainset, batch_size=batch_size, shuffle=True)
testdata = DataLoader(testset, batch_size=batch_size, shuffle=True)

### Método para avaliação

In [ ]:
from sklearn.metrics import f1_score

def evaluate(testdata):
  y_real, y_pred = [], []
  for batch_idx, row in enumerate(testdata):
    outputs = model(row['X'])
    y_pred.extend(torch.argmax(outputs, 1).tolist())

    y_real.extend([c2id[c] for c in row['y']])

  print('F1-Score:', f1_score(y_real, y_pred))

In [ ]:
for epoch in range(nepochs):
  losses = []
  for batch_idx, row in enumerate(traindata):
    X = row['X']
    y = torch.tensor([c2id[c] for c in row['y']])

    # Forward
    outputs = model(X)

    # Calculate loss
    loss = criterion(outputs, y)
    losses.append(float(loss))

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # Display
    if (batch_idx+1) % batch_status == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTotal Loss: {:.6f}'.format(
                            epoch+1, batch_idx + 1, len(traindata),
                            100. * batch_idx / len(traindata), float(loss),
                            round(sum(losses) / len(losses), 5)))

  evaluate(testdata)

Train Epoch: 1 [64/398 (16%)]	Loss: 0.550223	Total Loss: 0.663090
Train Epoch: 1 [128/398 (32%)]	Loss: 0.755304	Total Loss: 0.663030
Train Epoch: 1 [192/398 (48%)]	Loss: 0.762928	Total Loss: 0.660510
Train Epoch: 1 [256/398 (64%)]	Loss: 0.673048	Total Loss: 0.663490
Train Epoch: 1 [320/398 (80%)]	Loss: 0.637865	Total Loss: 0.661710
Train Epoch: 1 [384/398 (96%)]	Loss: 0.668283	Total Loss: 0.659640
F1-Score: 0.7569471624266145
Train Epoch: 2 [64/398 (16%)]	Loss: 0.683878	Total Loss: 0.655830
Train Epoch: 2 [128/398 (32%)]	Loss: 0.631069	Total Loss: 0.658840
Train Epoch: 2 [192/398 (48%)]	Loss: 0.648395	Total Loss: 0.657530
Train Epoch: 2 [256/398 (64%)]	Loss: 0.620742	Total Loss: 0.652760
Train Epoch: 2 [320/398 (80%)]	Loss: 0.658931	Total Loss: 0.654970
Train Epoch: 2 [384/398 (96%)]	Loss: 0.703110	Total Loss: 0.654590
F1-Score: 0.7564604541895067
Train Epoch: 3 [64/398 (16%)]	Loss: 0.677524	Total Loss: 0.616430
Train Epoch: 3 [128/398 (32%)]	Loss: 0.594190	Total Loss: 0.615890
Train E

### Predição

In [ ]:
val_X = ["Lucas", "Carlos", "Maria", "João"]

outputs = model(val_X)
val_y = torch.argmax(outputs, 1).tolist()

[id2c[c] for c in val_y]

['male', 'female', 'female', 'male']

# **Análise de Sentimentos** <br>
B2W Digital é uma empresa de comércio eletrônico criada no final de 2006 pela fusão entre Submarino, Shoptime, Americanas.<br>
Neste exemplo, usaremos um córpus de revisão de produtos nestas plataformas. O objetivo é treinar um classificador para prever se uma revisão é boa, neutra ou ruim.

In [ ]:
import torch
import torch.nn as nn
from torch import optim

## Baixando o córpus

In [ ]:
!wget https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/master/B2W-Reviews01.csv

--2023-08-08 15:02:07--  https://raw.githubusercontent.com/b2wdigital/b2w-reviews01/master/B2W-Reviews01.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49453175 (47M) [text/plain]
Saving to: ‘B2W-Reviews01.csv’

B2W-Reviews01.csv   100%[===================>]  47.16M   290MB/s    in 0.2s    

2023-08-08 15:02:09 (290 MB/s) - ‘B2W-Reviews01.csv’ saved [49453175/49453175]



## Carregando o córpus

In [ ]:
import csv
from random import shuffle

with open('B2W-Reviews01.csv') as f:
  reader = csv.reader(f, delimiter=',', quotechar='\"')
  corpus = list(reader)

  header, corpus = corpus[0], corpus[1:]

shuffle(corpus)

## Separando o córpus em conjuntos de treinamento e teste

In [ ]:
size = int(len(corpus) * 0.2)
treino = corpus[size:]
teste = corpus[:size]

len(treino), len(teste)

(105899, 26474)

## Extraindo as features <br>
Como features, será utilizado informações do produto, como nome, marca e categoria: assim como informações da revisão do produto, como título e conteúdo.<br>
A partir destas informações, o objetivo é prever se a revisão do produto é boa, neutra ou ruim.

In [ ]:
def get_features(info):
  product_id = info[2]
  product_name = info[3]
  product_brand = info[4]
  site_category_lv1 = info[5]
  site_category_lv2 = info[6]
  review_title = info[7]
  overall_rating = info[8]
  recommend_to_a_friend = info[9]
  review_text = info[10]
  reviewer_birth_year = info[11]
  reviewer_gender = info[12]
  reviewer_state = info[13]

  x = ' '.join([product_name,
                product_brand,
                site_category_lv1,
                site_category_lv2,
                review_title,
                review_text])
  y = 2 if overall_rating in ['4', '5'] else 0 if overall_rating in ['1', '2'] else 1
  return { 'X': x, 'y': y }

## Representação Vetorial TF-IDF

Assim como aprendemos na aula anterior, o texto de entrada com as informações do produto e da revisão será codificado num vetor TF-IDF.

Uma vez codificado, o vetor será passado para uma Regressão Logística que fará a classificação.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

def tokenize(texto):
  return nltk.word_tokenize(texto)

pipe = Pipeline([
  ('count', CountVectorizer(tokenizer=tokenize, stop_words=stopwords, min_df=5)),
  ('tfidf', TfidfTransformer()),
])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Treinando o modelo TF-IDF


In [ ]:
feat_treino = [get_features(w) for w in treino]
treino_X = [w['X'] for w in feat_treino]
treino_y = [w['y'] for w in feat_treino]

pipe.fit(treino_X, treino_y)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('count',
                 CountVectorizer(min_df=5,
                                 stop_words=['a', 'à', 'ao', 'aos', 'aquela',
                                             'aquelas', 'aquele', 'aqueles',
                                             'aquilo', 'as', 'às', 'até', 'com',
                                             'como', 'da', 'das', 'de', 'dela',
                                             'delas', 'dele', 'deles', 'depois',
                                             'do', 'dos', 'e', 'é', 'ela',
                                             'elas', 'ele', 'eles', ...],
                                 tokenizer=<function tokenize at 0x7eee88d31d80>)),
                ('tfidf', TfidfTransformer())])

In [ ]:
nvocab = pipe.transform(["Test"]).shape[1]
nvocab

21723

## Regressão Logística

In [ ]:
class SentimentPredictor(nn.Module):
  def __init__(self, inp_dim, hdim, n_classes, tfidfer):
    super(SentimentPredictor, self).__init__()
    self.tfidfer = tfidfer
    self.layer1 = nn.Linear(inp_dim, hdim)
    self.activation1 = nn.Sigmoid()
    self.layer2 = nn.Linear(hdim, n_classes)
    self.softmax = nn.LogSoftmax(1)

  def forward(self, X):
    tfidf = self.tfidfer.transform(X).toarray()
    tfidf = torch.tensor(tfidf).float()

    z1 = self.layer1(tfidf)
    a1 = self.activation1(z1)
    z2 = self.layer2(a1)
    return self.softmax(z2)

## Inicializando o Modelo

In [ ]:
inp_dim = nvocab
hdim = 1024
n_classes = len(set(treino_y))
model = SentimentPredictor(inp_dim, hdim, n_classes, pipe)

## Instanciando função de erro, otimizador e dados de treinamento e teste em lotes

In [ ]:
nepochs = 5
batch_size = 256
batch_status = 64
learning_rate = 0.01
criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
from torch.utils.data import DataLoader, Dataset

traindata = DataLoader([get_features(x) for x in treino], batch_size=batch_size, shuffle=True)
testdata = DataLoader([get_features(x) for x in teste], batch_size=batch_size, shuffle=True)

## Avaliação

In [ ]:
from sklearn.metrics import f1_score

def evaluate(testdata):
  y_real, y_pred = [], []
  for batch_idx, row in enumerate(testdata):
    outputs = model(row['X'])
    y_pred.extend(torch.argmax(outputs, 1).tolist())

    y_real.extend(row['y'])

  print('F1-Score:', f1_score(y_real, y_pred, average='weighted'))